In [15]:
import pandas as pd

import pandas as pd
import glob
cornbelters_files = glob.glob("../KCLData/*.csv")

all_files = cornbelters_files 

# Read and concatenate all CSVs into one DataFrame
xba_data_list = [pd.read_csv(f) for f in all_files]
df = pd.concat(xba_data_list, ignore_index=True)


# Filter for Normal CornBelters batters
df = df[df['BatterTeam'] == 'Kcl bluecaps 2025']

# Function to compute stats for a given pitcher handedness
def batter_stats_vs_pitcher(df, pitcher_hand):
    filtered = df[df['PitcherThrows'] == pitcher_hand].dropna(subset=['PlayResult'])
    # Plate appearances: count of unique PitchNo per batter (or use PAofInning if available)

    # Exclude rows that are not official at bats

    # Homeruns
    hr = filtered[filtered['PlayResult'] == 'HomeRun'].groupby(['Batter']).size().rename('homeruns')
    # Triples
    triples = filtered[filtered['PlayResult'] == 'Triple'].groupby(['Batter']).size().rename('triples')
    # Doubles
    doubles = filtered[filtered['PlayResult'] == 'Double'].groupby(['Batter']).size().rename('doubles')
    # Singles (PlayResult == 'Single')
    singles = filtered[filtered['PlayResult'] == 'Single'].groupby(['Batter']).size().rename('singles')
    # Walks (PitchCall == 'Walk' or 'IntentionalWalk')
    walks = filtered[filtered['KorBB'].isin(['Walk'])].groupby(['Batter']).size().rename('walks')
    # Outs (PlayResult == 'Out' or similar)
    outs = filtered[filtered['PlayResult'].isin(['StrikeoutLooking','StrikeoutSwinging','Out'])].groupby(['Batter']).size().rename('outs')

    # Assuming you have a DataFrame 'stats' with these columns for each batter
    
    # Combine all stats
    stats = pd.concat([hr, triples, doubles, singles, walks, outs], axis=1).fillna(0).reset_index()
    stats['at_bats'] = (
        stats['homeruns'] +
        stats['triples'] +
        stats['doubles'] +
        stats['singles'] +
        stats['outs']
    )
    # Define the player-to-position mapping
    cornbelters_player_position = {
        'Jackson Smith': 'if',
        'Jackson Stanek': 'if',
        'Tyler Thompson': 'if',
        'Nolan Mccrossin' : 'if',
        'Shea Zbrozek': 'if',
        'Jack Novak': 'of',
        'Kannan Kleine': 'c',
        'Caleb Royer': 'if',
        'Charlie Graham': 'c',
        'Cole Freeman': 'of',
        'Michael Kuska': 'of',
        'Ryan Colucci': 'if',
        'Zach Zychowski': 'if'
    }
    bobcats_player_position = {
    'Zach Bava': 'c',
    'Braxton Waller': 'c',
    'Nolan Hunter': 'c',
    'Jacob Trusner': 'if',
    'Trey Schilling': 'if',
    'Michael Lucas': 'if',
    'Jackson Natanek': 'if',
    'Cameron Anderson': 'if',
    'Jake Morrill': 'of',
    'Grayson Schnierle': 'of',
    'Riley Hendren': 'of',
    'Cal Darling': 'of',
    'Trey Blanchette': 'of',
    'Chase Peterson': 'of',
    'Andrew Schweigert': 'of'
}

    # Add a new 'position' column to your DataFrame based on the mapping
    stats['position'] = stats['Batter'].map(bobcats_player_position)
    stats['order'] = range(1, len(stats) + 1)
    stats['player_name'] = stats['Batter']
    stats = stats[['order','position', 'player_name', 'homeruns', 'triples', 'doubles', 'singles', 'walks', 'outs', 'at_bats']]
    cols = ['homeruns', 'triples', 'doubles', 'singles', 'walks', 'outs', 'at_bats']
    stats[cols] = stats[cols].fillna(0).astype(int)
    return stats

# For right-handed pitchers
stats_vs_right = batter_stats_vs_pitcher(df, 'Right')
# For left-handed pitchers
stats_vs_left = batter_stats_vs_pitcher(df, 'Left')

# Example: print both DataFrames
print("Vs Right-handed Pitchers:")
print(stats_vs_right)
print("\nVs Left-handed Pitchers:")
print(stats_vs_left)
stats_vs_left.to_csv('stats_vs_left.csv', index=False)
stats_vs_right.to_csv('stats_vs_right.csv', index=False)

Vs Right-handed Pitchers:
    order position        player_name  homeruns  triples  doubles  singles  \
0       1      NaN      Ethan Hamrick         1        0        1        1   
1       2       of     Chase Peterson         0        1        0        6   
2       3       of      Riley Hendren         0        1        0        3   
3       4        c     Braxton Waller         0        0        2        7   
4       5       of  Grayson Schnierle         0        0        2        4   
5       6      NaN     Josh Eisfelder         0        0        1        1   
6       7       if      Michael Lucas         0        0        1        3   
7       8        c       Nolan Hunter         0        0        1        4   
8       9       of    Trey Blanchette         0        0        2        8   
9      10       of  Andrew Schweigert         0        0        0        1   
10     11      NaN       Brandon Ward         0        0        0        2   
11     12       of        Cal Darling 